In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

In [3]:
batch_size = 32

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((.5, .5, .5), (.5, .5, .5))])

trainset = torchvision.datasets.CIFAR10(root = './data', download = True, train = True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root = './data', download = True, train = False, transform = transform)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = True, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [6]:
len(testloader)

313

In [7]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv2d_l1 = nn.Conv2d(3, 32, 3)
        self.bn_l1 = nn.BatchNorm2d(32)
        self.conv2d_l2 = nn.Conv2d(32, 32, 3)
        self.bn_l2 = nn.BatchNorm2d(32)
        self.maxpool_l2 = nn.MaxPool2d(2)
        self.dropout_l2 = nn.Dropout2d(0.2)
        
        self.conv2d_l3 = nn.Conv2d(32, 64, 3)
        self.bn_l3 = nn.BatchNorm2d(64)
        self.conv2d_l4 = nn.Conv2d(64, 64, 3)
        self.bn_l4 = nn.BatchNorm2d(64)
        self.maxpool_l4 = nn.MaxPool2d(2)
        self.dropout_l4 = nn.Dropout2d(0.3)
        
        self.conv2d_l5 = nn.Conv2d(64, 128, 3)
        self.bn_l5 = nn.BatchNorm2d(128)
        self.conv2d_l6 = nn.Conv2d(128, 128, 3)
        self.bn_l6 = nn.BatchNorm2d(128)
        self.maxpool_l6 = nn.MaxPool2d(2)
        self.dropout_l6 = nn.Dropout2d(0.4)
        
        self.fc1 = nn.Linear(2048, 512)
        self.fc2 = nn.Linear(512, 10)
        
        self.samePad = nn.ZeroPad2d(1)
        
    def forward(self, x):
        
        x = F.leaky_relu(self.bn_l1(self.conv2d_l1(self.samePad(x))))
        x = F.leaky_relu(self.bn_l2(self.conv2d_l2(self.samePad(x))))
        x = self.dropout_l2(self.maxpool_l2(x))
        
        x = F.leaky_relu(self.bn_l3(self.conv2d_l3(self.samePad(x))))
        x = F.leaky_relu(self.bn_l4(self.conv2d_l4(self.samePad(x))))
        x = self.dropout_l4(self.maxpool_l4(x))
        
        x = F.leaky_relu(self.bn_l5(self.conv2d_l5(self.samePad(x))))
        x = F.leaky_relu(self.bn_l6(self.conv2d_l6(self.samePad(x))))
        x = self.dropout_l6(self.maxpool_l6(x))
        
        x = x.view(-1,2048)
        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = F.log_softmax(x, dim=-1)
        
        return x
model = Net()      

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device('cpu')
print(device)

cuda:0


In [9]:
model = model.to(device)

In [10]:
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30,50,80], gamma=0.3)

In [11]:
model.train()

for epoch in range(85):
    
    running_loss = 0.0
    correct, total = 0, 0
    correct_epoch, total_epoch = 0, 0
    
    scheduler.step()
    
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        c = (torch.argmax(outputs,1) == labels)
        correct += torch.sum(c)
        total += 32
        correct_epoch += torch.sum(c)
        total_epoch += 32
        
        if i % 200 == 199:    # print every 200 mini-batches
            print('[%d, %5d] loss: %.7f accuracy: [%d, %d] (%d %%)' % (epoch + 1, i + 1, running_loss / 200, correct, total, 100*correct/total))
            running_loss = 0.0
            correct = 0
            total = 0
    
    print('<=========== accuracy after {} epoch is [{:d} {:d}] ({:d} %%) ===========>)'.format(epoch+1, correct_epoch, total_epoch, 100*correct_epoch/total_epoch))

print('Finished Training')


[1,   200] loss: 2.1884397 accuracy: [1297, 6400] (20 %)
[1,   400] loss: 1.8997445 accuracy: [2005, 6400] (31 %)
[1,   600] loss: 1.6916031 accuracy: [2416, 6400] (37 %)
[1,   800] loss: 1.5882624 accuracy: [2632, 6400] (41 %)
[1,  1000] loss: 1.5244598 accuracy: [2775, 6400] (43 %)
[1,  1200] loss: 1.4405664 accuracy: [3032, 6400] (47 %)


KeyboardInterrupt: 

In [63]:
torch.save(model.state_dict(), 'mymodel')